<a href="https://colab.research.google.com/github/Kaleab1999/NER/blob/main/NER_1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm #to view the progress

nlp = spacy.load("en_core_web_sm") # load a new spacy model
db = DocBin() # create a DocBin object

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import json
f = open('/content/drive/MyDrive/dataset/annotations.json')
TRAIN_DATA = json.load(f)

In [5]:
TRAIN_DATA

{'classes': ['NAME', 'PHONE NUMBER', 'LOCATION', 'UNIVERSITY'],
 'annotations': [['My name is Kaleab. I am from Ethiopia. Every Saturday @8 pm you can contact me at +251930964236. 930964236\r',
   {'entities': [[11, 18, 'NAME'],
     [29, 38, 'LOCATION'],
     [82, 96, 'PHONE NUMBER'],
     [97, 106, 'PHONE NUMBER']]}],
  ["First of all, thank you so much for giving me this opportunity to introduce myself. My name is Sultan Haider. I'm 21 years old. I'm from Hanumangarh JN. Which is located in Rajasthan. I completed graduation in 2021 and passed 12th on the RBSE board. I have played games from the age of 4 years and achieved one state-level certificate. I'm fresher, I'm a multitasker, a good communicator, and a positive thinker. 1-541-754-3010I am Shalini.\r",
   {'entities': [[95, 109, 'NAME'],
     [137, 152, 'LOCATION'],
     [173, 183, 'LOCATION'],
     [413, 428, 'PHONE NUMBER'],
     [432, 439, 'NAME']]}],
  ["I am from Visakhapatnam. I have done my schooling at Z.P.H school in S

In [6]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 8/8 [00:00<00:00, 453.16it/s]


In [7]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2022-10-26 04:29:28,784] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-10-26 04:29:28,795] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-10-26 04:29:28,799] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/spacy/language.py", line 1302, in initialize
    self, data=I["vocab_data"], lookups=I["lookups"], vectors=I["vectors"]
  File "/usr/local/lib/python3.7/dist-packages/spacy/training/initialize.py", line 131, in init_vocab
    load_vectors_into_model(nlp, vectors)
  File "/usr/local/lib/python3.7/dist-packages/spacy/training/initialize.py", line 152, in load_vectors_into_model
    vectors_nlp = load_model(name, vocab=nlp.vocab, exclude=exclude)
  File "/u

In [9]:
nlp_ner = spacy.load("/content/model-best")

In [36]:
doc= nlp_ner('''I'm from Kaleab Wondemu Nuri from Ethiopia. Every Saturday @8 pm you can contact me at +251930964236. 
930964236.
I graduated BTech from university of Gondar at Bhubaneswar.
''')

In [37]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter